# Simple PyGDF Example

Redo the Simple Numba example to use PyGDF
This Example uses a smaller dataset and combines a number of the functions into a single operation

In [ ]:
# This automatically time every cell's execution
!pip install ipython-autotime
%load_ext autotime

In [ ]:
#Imports 
import numpy as np 
import pandas as pd
import math

import pygdf 
import numba as nb
from numba import vectorize, jit

In [ ]:
# Read in data and adjust
pdf = pd.read_csv("./data/1B.csv")
pdf.shape

In [ ]:
pdf.size

In [ ]:
# Convert to a GPU DataFrame
gdf = pygdf.DataFrame.from_pandas(pdf)

### Scale the data so that it is in the range of -200 to +200
This will also create a new column name

In [ ]:
# Scale and Converet the data
def ScaleData( Data, B, Scale) :
    for i in range (Data.size) :
        B[i] = Data[i] * Scale

In [ ]:
gdf = gdf.apply_rows(ScaleData, incols=['Data'], outcols=dict(B=np.float32), kwargs=dict(Scale=200))

In [ ]:
gdf.head()

In [ ]:
gdf.drop_column('Data')

In [ ]:
gdf.head()

### Compute the Mean and Standard Deviation

In [ ]:
mean = gdf['B'].mean()
std = gdf['B'].std()
print("Std == %f \t Mean == %f" % (std, mean))

### Normalize

In [ ]:
def NormalizeData( B, C, s, m) :
    for i in range(B.size):
        C[i] = math.floor((B[i] - m)/s)

In [ ]:
gdf = gdf.apply_rows(NormalizeData, incols=['B'], outcols=dict(C=np.float32), kwargs=dict(s=std, m=mean ))

In [ ]:
gdf.drop_column('B')
gdf.head()

# Group and count

In [ ]:
from collections import OrderedDict

aggs = OrderedDict()
aggs['C'] = 'count'

stats = gdf.groupby(['C']).agg(aggs)


In [ ]:
stats